In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier,StackingClassifier, HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, KBinsDiscretizer, RobustScaler
from sklearn.model_selection import KFold,StratifiedKFold, GroupKFold,train_test_split
import gc
import datetime
from tqdm.notebook import tqdm ,tnrange
import warnings
warnings.filterwarnings("ignore")
np.random.seed(0)
pd.set_option('display.max_columns', 500)

In [2]:
## Loading Train and Test data
df_train = pd.read_csv("train_s3TEQDk.csv")
df_test = pd.read_csv("test_mSzZ8RL.csv")
## there are some rows where all values expect id were same including the target (Is_Lead), dropping them
df_train = df_train.drop_duplicates(subset=[col for col in df_train.columns if 'ID' not in col])
## there are some observations where although all features had same values but target was different, so dropped those too
df_train = df_train[~df_train.duplicated(subset = [i for i in list(df_train.columns) if 'Is_Lead' not in i and 'ID' not in i], keep=False)].sort_values(list(df_train.columns)).reset_index(drop=True)
df_total = pd.concat([df_train,df_test],ignore_index=True, sort=False)
print("Shape of datasets: ", df_train.shape, df_test.shape, df_total.shape)

Shape of datasets:  (245700, 11) (105312, 10) (351012, 11)


In [3]:
## filling null values in credit column with a string now, but lets see if we can predict this column using other features later.
## predicting didnot work
df_total['Credit_Product'].fillna("""Missing""", inplace = True)

## one idea that just struck my mind is converting this to a text classification problem,
## just so that we can used large bert architectures. Lets see if I work on that.

## Statement will be: will a 23 year old, financially active, self employed female living in RG282,
## who is associated with the company for 14 years and already has a credit product with 1005068
## bank balance be interested in buying a credit card if contacted through X2 channel?

In [4]:
df_total

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,222A8XWS,Male,66,RG280,Other,X2,50,No,819609,No,0.0
1,222HSZEH,Male,49,RG268,Self_Employed,X3,69,Yes,679666,No,0.0
2,222TDSNN,Male,32,RG262,Salaried,X1,32,No,761982,No,0.0
3,224FPNSD,Female,39,RG276,Self_Employed,X2,26,Missing,583519,Yes,1.0
4,224VSEND,Male,29,RG261,Salaried,X1,13,No,736866,Yes,0.0
...,...,...,...,...,...,...,...,...,...,...,...
351007,DBENJOYI,Male,52,RG268,Salaried,X2,86,Yes,4242558,Yes,NaN
351008,CWQ72DWS,Male,55,RG277,Other,X2,86,Yes,1159153,No,NaN
351009,HDESC8GU,Male,35,RG254,Salaried,X4,15,No,1703727,No,NaN
351010,2PW4SFCA,Male,53,RG254,Other,X3,93,No,737178,Yes,NaN


### Feature Engineering

In [5]:
col_discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
for col in['Age','Vintage','Avg_Account_Balance']:
    df_total[f'{col}_Bins'] =col_discretizer.fit_transform(df_total[f'{col}'].values.reshape(-1,1)).astype(int)

In [6]:
for col1 in ['Gender','Region_Code','Occupation','Channel_Code','Is_Active','Age_Bins','Vintage_Bins','Avg_Account_Balance_Bins']:
    df_total[f'Credit_Product_' + f'{col1}' ] = df_total[f'Credit_Product'].astype(str)+'_'+df_total[f'{col1}'].astype(str)

for col1 in ['Gender','Region_Code','Channel_Code','Is_Active','Age_Bins','Vintage_Bins','Avg_Account_Balance_Bins']:
    df_total[f'Occupation_' + f'{col1}' ] = df_total[f'Occupation'].astype(str)+'_'+df_total[f'{col1}'].astype(str)
    
for col1 in ['Gender','Region_Code','Is_Active','Age_Bins','Vintage_Bins','Avg_Account_Balance_Bins']:
    df_total[f'Channel_Code_' + f'{col1}' ] = df_total[f'Channel_Code'].astype(str)+'_'+df_total[f'{col1}'].astype(str)
    
for col1 in ['Gender','Region_Code','Age_Bins','Vintage_Bins','Avg_Account_Balance_Bins']:
    df_total[f'Is_Active_' + f'{col1}' ] = df_total[f'Is_Active'].astype(str)+'_'+df_total[f'{col1}'].astype(str)
    
for col1 in ['Gender','Age_Bins','Vintage_Bins','Avg_Account_Balance_Bins']:
    df_total[f'Region_Code_' + f'{col1}' ] = df_total[f'Region_Code'].astype(str)+'_'+df_total[f'{col1}'].astype(str)
    
for col1 in ['Gender','Region_Code','Channel_Code','Is_Active','Age_Bins','Vintage_Bins','Avg_Account_Balance_Bins']:
    df_total[f'Credit_Product_' + f'Occupation_' + f'{col1}' ] = df_total[f'Credit_Product'].astype(str)+'_'+df_total[f'Occupation'].astype(str)+"_"+df_total[f'{col1}'].astype(str)

In [7]:
freq_encoding_features = [col for col in df_total.columns if 'ID' not in col and 'Is_Lead' not in col]
print(freq_encoding_features,'\n')

categorical_agg_features = [col for col in df_total.columns if df_total[col].dtype=='O' and 'ID' not in col]
print(categorical_agg_features)

['Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code', 'Vintage', 'Credit_Product', 'Avg_Account_Balance', 'Is_Active', 'Age_Bins', 'Vintage_Bins', 'Avg_Account_Balance_Bins', 'Credit_Product_Gender', 'Credit_Product_Region_Code', 'Credit_Product_Occupation', 'Credit_Product_Channel_Code', 'Credit_Product_Is_Active', 'Credit_Product_Age_Bins', 'Credit_Product_Vintage_Bins', 'Credit_Product_Avg_Account_Balance_Bins', 'Occupation_Gender', 'Occupation_Region_Code', 'Occupation_Channel_Code', 'Occupation_Is_Active', 'Occupation_Age_Bins', 'Occupation_Vintage_Bins', 'Occupation_Avg_Account_Balance_Bins', 'Channel_Code_Gender', 'Channel_Code_Region_Code', 'Channel_Code_Is_Active', 'Channel_Code_Age_Bins', 'Channel_Code_Vintage_Bins', 'Channel_Code_Avg_Account_Balance_Bins', 'Is_Active_Gender', 'Is_Active_Region_Code', 'Is_Active_Age_Bins', 'Is_Active_Vintage_Bins', 'Is_Active_Avg_Account_Balance_Bins', 'Region_Code_Gender', 'Region_Code_Age_Bins', 'Region_Code_Vintage_Bins', 'Region_

In [8]:
for col in freq_encoding_features:
    val_counts = df_total[col].value_counts().to_dict()
    df_total[str(col)+"_counts"] = df_total[col].map(val_counts)

In [9]:
for col1 in categorical_agg_features:
    for col2 in categorical_agg_features:
        if col1!=col2:
            df_total['unique_of_'+str(col2)+'_per_'+str(col1)] = df_total.groupby([col1])[col2].transform('nunique')
            
for col1 in categorical_agg_features:
    for col2 in ['Age','Vintage','Avg_Account_Balance']:
        df_total['mean_of_'+str(col2)+'_per_'+str(col1)] = df_total.groupby([col1])[col2].transform('mean')
        df_total['sum_of_'+str(col2)+'_per_'+str(col1)] = df_total.groupby([col1])[col2].transform('sum')
        df_total['std_of_'+str(col2)+'_per_'+str(col1)] = df_total.groupby([col1])[col2].transform('std').fillna(-1)

In [10]:
df_total.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,Age_Bins,Vintage_Bins,Avg_Account_Balance_Bins,Credit_Product_Gender,Credit_Product_Region_Code,Credit_Product_Occupation,Credit_Product_Channel_Code,Credit_Product_Is_Active,Credit_Product_Age_Bins,Credit_Product_Vintage_Bins,Credit_Product_Avg_Account_Balance_Bins,Occupation_Gender,Occupation_Region_Code,Occupation_Channel_Code,Occupation_Is_Active,Occupation_Age_Bins,Occupation_Vintage_Bins,Occupation_Avg_Account_Balance_Bins,Channel_Code_Gender,Channel_Code_Region_Code,Channel_Code_Is_Active,Channel_Code_Age_Bins,Channel_Code_Vintage_Bins,Channel_Code_Avg_Account_Balance_Bins,Is_Active_Gender,Is_Active_Region_Code,Is_Active_Age_Bins,Is_Active_Vintage_Bins,Is_Active_Avg_Account_Balance_Bins,Region_Code_Gender,Region_Code_Age_Bins,Region_Code_Vintage_Bins,Region_Code_Avg_Account_Balance_Bins,Credit_Product_Occupation_Gender,Credit_Product_Occupation_Region_Code,Credit_Product_Occupation_Channel_Code,Credit_Product_Occupation_Is_Active,Credit_Product_Occupation_Age_Bins,Credit_Product_Occupation_Vintage_Bins,Credit_Product_Occupation_Avg_Account_Balance_Bins,Gender_counts,Age_counts,Region_Code_counts,Occupation_counts,Channel_Code_counts,Vintage_counts,Credit_Product_counts,Avg_Account_Balance_counts,Is_Active_counts,Age_Bins_counts,Vintage_Bins_counts,Avg_Account_Balance_Bins_counts,Credit_Product_Gender_counts,Credit_Product_Region_Code_counts,Credit_Product_Occupation_counts,Credit_Product_Channel_Code_counts,Credit_Product_Is_Active_counts,Credit_Product_Age_Bins_counts,Credit_Product_Vintage_Bins_counts,Credit_Product_Avg_Account_Balance_Bins_counts,Occupation_Gender_counts,Occupation_Region_Code_counts,Occupation_Channel_Code_counts,Occupation_Is_Active_counts,Occupation_Age_Bins_counts,Occupation_Vintage_Bins_counts,Occupation_Avg_Account_Balance_Bins_counts,Channel_Code_Gender_counts,Channel_Code_Region_Code_counts,Channel_Code_Is_Active_counts,Channel_Code_Age_Bins_counts,Channel_Code_Vintage_Bins_counts,Channel_Code_Avg_Account_Balance_Bins_counts,Is_Active_Gender_counts,Is_Active_Region_Code_counts,Is_Active_Age_Bins_counts,Is_Active_Vintage_Bins_counts,Is_Active_Avg_Account_Balance_Bins_counts,Region_Code_Gender_counts,Region_Code_Age_Bins_counts,Region_Code_Vintage_Bins_counts,Region_Code_Avg_Account_Balance_Bins_counts,Credit_Product_Occupation_Gender_counts,Credit_Product_Occupation_Region_Code_counts,Credit_Product_Occupation_Channel_Code_counts,Credit_Product_Occupation_Is_Active_counts,Credit_Product_Occupation_Age_Bins_counts,Credit_Product_Occupation_Vintage_Bins_counts,Credit_Product_Occupation_Avg_Account_Balance_Bins_counts,unique_of_Region_Code_per_Gender,unique_of_Occupation_per_Gender,unique_of_Channel_Code_per_Gender,unique_of_Credit_Product_per_Gender,unique_of_Is_Active_per_Gender,unique_of_Credit_Product_Gender_per_Gender,unique_of_Credit_Product_Region_Code_per_Gender,unique_of_Credit_Product_Occupation_per_Gender,unique_of_Credit_Product_Channel_Code_per_Gender,unique_of_Credit_Product_Is_Active_per_Gender,unique_of_Credit_Product_Age_Bins_per_Gender,unique_of_Credit_Product_Vintage_Bins_per_Gender,unique_of_Credit_Product_Avg_Account_Balance_Bins_per_Gender,unique_of_Occupation_Gender_per_Gender,unique_of_Occupation_Region_Code_per_Gender,unique_of_Occupation_Channel_Code_per_Gender,unique_of_Occupation_Is_Active_per_Gender,unique_of_Occupation_Age_Bins_per_Gender,unique_of_Occupation_Vintage_Bins_per_Gender,unique_of_Occupation_Avg_Account_Balance_Bins_per_Gender,unique_of_Channel_Code_Gender_per_Gender,unique_of_Channel_Code_Region_Code_per_Gender,unique_of_Channel_Code_Is_Active_per_Gender,unique_of_Channel_Code_Age_Bins_per_Gender,unique_of_Channel_Code_Vintage_Bins_per_Gender,unique_of_Channel_Code_Avg_Account_Balance_Bins_per_Gender,unique_of_Is_Active_Gender_per_Gender,unique_of_Is_Active_Region_Code_per_Gender,unique_of_Is_Active_Age_Bins_per_Gender,unique_of_Is_Active_Vinta

In [11]:
remove_non_unique_cols = []
for col in df_total.columns:
    if df_total[col].nunique()<=1:
        remove_non_unique_cols.append(col)
        
print(len(remove_non_unique_cols))

701


In [12]:
df_total.drop(columns=remove_non_unique_cols, axis=1, inplace=True)

In [13]:
df_total.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,Age_Bins,Vintage_Bins,Avg_Account_Balance_Bins,Credit_Product_Gender,Credit_Product_Region_Code,Credit_Product_Occupation,Credit_Product_Channel_Code,Credit_Product_Is_Active,Credit_Product_Age_Bins,Credit_Product_Vintage_Bins,Credit_Product_Avg_Account_Balance_Bins,Occupation_Gender,Occupation_Region_Code,Occupation_Channel_Code,Occupation_Is_Active,Occupation_Age_Bins,Occupation_Vintage_Bins,Occupation_Avg_Account_Balance_Bins,Channel_Code_Gender,Channel_Code_Region_Code,Channel_Code_Is_Active,Channel_Code_Age_Bins,Channel_Code_Vintage_Bins,Channel_Code_Avg_Account_Balance_Bins,Is_Active_Gender,Is_Active_Region_Code,Is_Active_Age_Bins,Is_Active_Vintage_Bins,Is_Active_Avg_Account_Balance_Bins,Region_Code_Gender,Region_Code_Age_Bins,Region_Code_Vintage_Bins,Region_Code_Avg_Account_Balance_Bins,Credit_Product_Occupation_Gender,Credit_Product_Occupation_Region_Code,Credit_Product_Occupation_Channel_Code,Credit_Product_Occupation_Is_Active,Credit_Product_Occupation_Age_Bins,Credit_Product_Occupation_Vintage_Bins,Credit_Product_Occupation_Avg_Account_Balance_Bins,Gender_counts,Age_counts,Region_Code_counts,Occupation_counts,Channel_Code_counts,Vintage_counts,Credit_Product_counts,Avg_Account_Balance_counts,Is_Active_counts,Age_Bins_counts,Vintage_Bins_counts,Avg_Account_Balance_Bins_counts,Credit_Product_Gender_counts,Credit_Product_Region_Code_counts,Credit_Product_Occupation_counts,Credit_Product_Channel_Code_counts,Credit_Product_Is_Active_counts,Credit_Product_Age_Bins_counts,Credit_Product_Vintage_Bins_counts,Credit_Product_Avg_Account_Balance_Bins_counts,Occupation_Gender_counts,Occupation_Region_Code_counts,Occupation_Channel_Code_counts,Occupation_Is_Active_counts,Occupation_Age_Bins_counts,Occupation_Vintage_Bins_counts,Occupation_Avg_Account_Balance_Bins_counts,Channel_Code_Gender_counts,Channel_Code_Region_Code_counts,Channel_Code_Is_Active_counts,Channel_Code_Age_Bins_counts,Channel_Code_Vintage_Bins_counts,Channel_Code_Avg_Account_Balance_Bins_counts,Is_Active_Gender_counts,Is_Active_Region_Code_counts,Is_Active_Age_Bins_counts,Is_Active_Vintage_Bins_counts,Is_Active_Avg_Account_Balance_Bins_counts,Region_Code_Gender_counts,Region_Code_Age_Bins_counts,Region_Code_Vintage_Bins_counts,Region_Code_Avg_Account_Balance_Bins_counts,Credit_Product_Occupation_Gender_counts,Credit_Product_Occupation_Region_Code_counts,Credit_Product_Occupation_Channel_Code_counts,Credit_Product_Occupation_Is_Active_counts,Credit_Product_Occupation_Age_Bins_counts,Credit_Product_Occupation_Vintage_Bins_counts,Credit_Product_Occupation_Avg_Account_Balance_Bins_counts,unique_of_Credit_Product_Occupation_per_Gender,unique_of_Occupation_Region_Code_per_Gender,unique_of_Credit_Product_Occupation_Gender_per_Gender,unique_of_Credit_Product_Occupation_Region_Code_per_Gender,unique_of_Credit_Product_Occupation_Channel_Code_per_Gender,unique_of_Credit_Product_Occupation_Is_Active_per_Gender,unique_of_Credit_Product_Occupation_Age_Bins_per_Gender,unique_of_Credit_Product_Occupation_Vintage_Bins_per_Gender,unique_of_Credit_Product_Occupation_Avg_Account_Balance_Bins_per_Gender,unique_of_Credit_Product_Occupation_per_Region_Code,unique_of_Occupation_Gender_per_Region_Code,unique_of_Occupation_Channel_Code_per_Region_Code,unique_of_Occupation_Is_Active_per_Region_Code,unique_of_Occupation_Age_Bins_per_Region_Code,unique_of_Occupation_Vintage_Bins_per_Region_Code,unique_of_Occupation_Avg_Account_Balance_Bins_per_Region_Code,unique_of_Channel_Code_Age_Bins_per_Region_Code,unique_of_Channel_Code_Vintage_Bins_per_Region_Code,unique_of_Channel_Code_Avg_Account_Balance_Bins_per_Region_Code,unique_of_Credit_Product_Occupation_Gender_per_Region_Code,unique_of_Credit_Product_Occupation_Region_Code_per_Region_Code,unique_of_Credit_Product_Occupation_Channel_Code_per_Region_Code,unique_of_Credit_Product_Occupation_Is_Active_per_Region_Code,unique_of_Credit_Pr

### Preparing Train and Test

In [14]:
train_x_full = df_total[df_total['Is_Lead'].notnull()].reset_index(drop=True)
test_x_full = df_total[df_total['Is_Lead'].isnull()].reset_index(drop=True)
train_y_full = train_x_full['Is_Lead']
del test_x_full['Is_Lead']
del train_x_full['Is_Lead']
## storing test_id to use while submitting solution
test_id = test_x_full['ID']
## del id column from train and test as they have all unique values
del train_x_full['ID']
del test_x_full['ID']
print(train_x_full.shape, train_y_full.shape, test_x_full.shape)

(245700, 1590) (245700,) (105312, 1590)


In [15]:
remove_non_unique_cols_train = []
for col in train_x_full.columns:
    if train_x_full[col].nunique()<=1:
        remove_non_unique_cols_train.append(col)
        
print(len(remove_non_unique_cols_train))

0


In [16]:
remove_non_unique_cols_test = []
for col in test_x_full.columns:
    if test_x_full[col].nunique()<=1:
        remove_non_unique_cols_test.append(col)
        
print(len(remove_non_unique_cols_test))

97


In [17]:
remove_non_unique_cols_based_on_train_test = list(set(set(remove_non_unique_cols_train).union(set(remove_non_unique_cols_test))))

In [18]:
df_total.drop(columns=remove_non_unique_cols_based_on_train_test, axis=1, inplace=True)

In [19]:
train_x_full = df_total[df_total['Is_Lead'].notnull()].reset_index(drop=True)
test_x_full = df_total[df_total['Is_Lead'].isnull()].reset_index(drop=True)
train_y_full = train_x_full['Is_Lead']
del test_x_full['Is_Lead']
del train_x_full['Is_Lead']
## storing test_id to use while submitting solution
test_id = test_x_full['ID']
## del id column from train and test as they have all unique values
del train_x_full['ID']
del test_x_full['ID']
print(train_x_full.shape, train_y_full.shape, test_x_full.shape)

(245700, 1493) (245700,) (105312, 1493)


In [20]:
cat_features = [col for col in train_x_full.columns if train_x_full[col].dtype=='O']
print(cat_features)

['Gender', 'Region_Code', 'Occupation', 'Channel_Code', 'Credit_Product', 'Is_Active', 'Credit_Product_Gender', 'Credit_Product_Region_Code', 'Credit_Product_Occupation', 'Credit_Product_Channel_Code', 'Credit_Product_Is_Active', 'Credit_Product_Age_Bins', 'Credit_Product_Vintage_Bins', 'Credit_Product_Avg_Account_Balance_Bins', 'Occupation_Gender', 'Occupation_Region_Code', 'Occupation_Channel_Code', 'Occupation_Is_Active', 'Occupation_Age_Bins', 'Occupation_Vintage_Bins', 'Occupation_Avg_Account_Balance_Bins', 'Channel_Code_Gender', 'Channel_Code_Region_Code', 'Channel_Code_Is_Active', 'Channel_Code_Age_Bins', 'Channel_Code_Vintage_Bins', 'Channel_Code_Avg_Account_Balance_Bins', 'Is_Active_Gender', 'Is_Active_Region_Code', 'Is_Active_Age_Bins', 'Is_Active_Vintage_Bins', 'Is_Active_Avg_Account_Balance_Bins', 'Region_Code_Gender', 'Region_Code_Age_Bins', 'Region_Code_Vintage_Bins', 'Region_Code_Avg_Account_Balance_Bins', 'Credit_Product_Occupation_Gender', 'Credit_Product_Occupation_Re

### CATBOOST

In [21]:
# Final ROC-AUC = 0.8742
train_x = train_x_full.copy()
train_y = train_y_full.copy()
test_x = test_x_full.copy()
feature_importance_df = pd.DataFrame()

NFOLDS = 5
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=18121995)
pred_test = []
pred_x = []
pred_y = []
j=1
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    model = CatBoostClassifier(n_estimators=20000,random_state=1812195,learning_rate=0.03,eval_metric='AUC',
                              cat_features =cat_features)
    model.fit(train_x.loc[train_ids], train_y.loc[train_ids], 
              eval_set=[(train_x.loc[train_ids], train_y.loc[train_ids]), (train_x.loc[test_ids], train_y.loc[test_ids])],
              verbose=500,
              early_stopping_rounds=200)
    pred_fold = model.predict_proba(train_x.loc[test_ids])[:,-1]
    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    pred_fold_test = model.predict_proba(test_x)[:,-1]
    pred_test.append([float(i) for i  in pred_fold_test])
    print('\n')
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_x.loc[train_ids].columns
    fold_importance_df["importance"] = model.feature_importances_
    fold_importance_df["fold"] = j + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    j=j+1

print("Final ROC-AUC Score:", roc_auc_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

vote = final_prediction.mean(axis=1)
final_prediction['Is_Lead'] = vote
final_prediction['ID'] = test_id.values
print(final_prediction.shape)

train_pred_all = pd.DataFrame()
train_pred_all['Is_Lead'] = pred_y
train_pred_all['cb_pred'] = pred_x
final_prediction_cb = final_prediction.copy()

final_prediction.head()

● Fold : 1
0:	test: 0.8628158	test1: 0.8575245	best: 0.8575245 (0)	total: 1.3s	remaining: 7h 12m 46s
500:	test: 0.8788127	test1: 0.8707141	best: 0.8707141 (500)	total: 9m 45s	remaining: 6h 19m 58s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.870818275
bestIteration = 763

Shrink model to first 764 iterations.


● Fold : 2
0:	test: 0.8614734	test1: 0.8628354	best: 0.8628354 (0)	total: 1.22s	remaining: 6h 46m 44s
500:	test: 0.8777185	test1: 0.8752544	best: 0.8752610 (499)	total: 9m 35s	remaining: 6h 13m 14s
1000:	test: 0.8805299	test1: 0.8755445	best: 0.8755445 (1000)	total: 19m 46s	remaining: 6h 15m 15s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8755903059
bestIteration = 1112

Shrink model to first 1113 iterations.


● Fold : 3
0:	test: 0.8620261	test1: 0.8606954	best: 0.8606954 (0)	total: 1.25s	remaining: 6h 55m 21s
500:	test: 0.8778105	test1: 0.8736808	best: 0.8736814 (496)	total: 10m 19s	remaining: 6h 42m
1000:	test: 0.8803611	test1:

,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,Is_Lead,ID
0,0.047243,0.044791,0.038201,0.040695,0.036415,0.041469,VBENBARO
1,0.877784,0.870863,0.879267,0.885380,0.889843,0.880627,CCMEWNKY
2,0.050817,0.055701,0.055668,0.051307,0.052312,0.053161,VK3KGA9M
3,0.023610,0.023294,0.025823,0.024535,0.021965,0.023845,TT8RPZVC
4,0.024540,0.023651,0.025000,0.024406,0.021039,0.023727,SHQZEYTZ


In [23]:
df_submission = final_prediction[['ID', 'Is_Lead']]
df_submission.to_csv("cb_v3.csv", index = False)
df_submission.head()

,ID,Is_Lead
0,VBENBARO,0.041469
1,CCMEWNKY,0.880627
2,VK3KGA9M,0.053161
3,TT8RPZVC,0.023845
4,SHQZEYTZ,0.023727


In [22]:
all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
all_features.reset_index(inplace=True)
important_features = list(all_features[all_features['importance']!=0]['feature'])
all_features

,feature,importance
0,unique_of_Occupation_Age_Bins_per_Credit_Produ...,4.179025
1,std_of_Age_per_Credit_Product_Is_Active,3.718169
2,Occupation_Age_Bins,3.035632
3,sum_of_Age_per_Credit_Product_Is_Active,2.929656
4,mean_of_Vintage_per_Credit_Product_Is_Active,2.747740
...,...,...
1488,unique_of_Occupation_Avg_Account_Balance_Bins_...,0.000000
1489,unique_of_Credit_Product_Gender_per_Region_Cod...,0.000000
1490,unique_of_Credit_Product_Is_Active_per_Channel...,0.000000
1491,unique_of_Occupation_Avg_Account_Balance_Bins_...,0.000000


In [29]:
imp_features_v1 = list(all_features[all_features['importance']>0.1]['feature'].values)

In [38]:
train_x_full_v1 = train_x_full[imp_features_v1]
test_x_full_v1 = test_x_full[imp_features_v1]

cat_features_v1 = [col for col in train_x_full_v1.columns if train_x_full_v1[col].dtype=='O']
print(cat_features_v1)

['Occupation_Age_Bins', 'Credit_Product_Occupation_Age_Bins', 'Credit_Product_Occupation_Vintage_Bins', 'Channel_Code_Vintage_Bins', 'Credit_Product_Channel_Code', 'Credit_Product_Vintage_Bins', 'Credit_Product_Occupation_Channel_Code', 'Credit_Product_Age_Bins', 'Credit_Product_Is_Active', 'Credit_Product', 'Occupation_Vintage_Bins', 'Is_Active_Vintage_Bins', 'Credit_Product_Occupation_Is_Active', 'Channel_Code_Is_Active', 'Channel_Code', 'Credit_Product_Occupation', 'Region_Code_Vintage_Bins', 'Credit_Product_Gender', 'Occupation', 'Occupation_Channel_Code', 'Occupation_Is_Active', 'Channel_Code_Age_Bins', 'Occupation_Gender', 'Credit_Product_Region_Code', 'Is_Active_Age_Bins', 'Channel_Code_Avg_Account_Balance_Bins']


In [39]:
# Final ROC-AUC = 0.8742
train_x = train_x_full_v1.copy()
train_y = train_y_full.copy()
test_x = test_x_full_v1.copy()
feature_importance_df = pd.DataFrame()

NFOLDS = 5
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=18121995)
pred_test = []
pred_x = []
pred_y = []
j=1
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    model = CatBoostClassifier(n_estimators=20000,random_state=1812195,learning_rate=0.03,eval_metric='AUC',
                              cat_features =cat_features_v1)
    model.fit(train_x.loc[train_ids], train_y.loc[train_ids], 
              eval_set=[(train_x.loc[train_ids], train_y.loc[train_ids]), (train_x.loc[test_ids], train_y.loc[test_ids])],
              verbose=500,
              early_stopping_rounds=200)
    pred_fold = model.predict_proba(train_x.loc[test_ids])[:,-1]
    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    pred_fold_test = model.predict_proba(test_x)[:,-1]
    pred_test.append([float(i) for i  in pred_fold_test])
    print('\n')
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_x.loc[train_ids].columns
    fold_importance_df["importance"] = model.feature_importances_
    fold_importance_df["fold"] = j + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    j=j+1

print("Final ROC-AUC Score:", roc_auc_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

vote = final_prediction.mean(axis=1)
final_prediction['Is_Lead'] = vote
final_prediction['ID'] = test_id.values
print(final_prediction.shape)

train_pred_all = pd.DataFrame()
train_pred_all['Is_Lead'] = pred_y
train_pred_all['cb_pred'] = pred_x
final_prediction_cb = final_prediction.copy()

final_prediction.head()

● Fold : 1
0:	test: 0.8645757	test1: 0.8582878	best: 0.8582878 (0)	total: 555ms	remaining: 3h 5m 4s
500:	test: 0.8778492	test1: 0.8713279	best: 0.8713403 (498)	total: 5m 27s	remaining: 3h 32m 25s
1000:	test: 0.8799788	test1: 0.8716354	best: 0.8716591 (902)	total: 10m 57s	remaining: 3h 28m 4s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8716590531
bestIteration = 902

Shrink model to first 903 iterations.


● Fold : 2
0:	test: 0.8630838	test1: 0.8642857	best: 0.8642857 (0)	total: 685ms	remaining: 3h 48m 27s
500:	test: 0.8767686	test1: 0.8754662	best: 0.8754718 (497)	total: 5m 34s	remaining: 3h 36m 49s
1000:	test: 0.8787959	test1: 0.8756672	best: 0.8756831 (846)	total: 11m 11s	remaining: 3h 32m 21s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8757238657
bestIteration = 1153

Shrink model to first 1154 iterations.


● Fold : 3
0:	test: 0.8630922	test1: 0.8630889	best: 0.8630889 (0)	total: 745ms	remaining: 4h 8m 25s
500:	test: 0.8769165	test1

,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,Is_Lead,ID
0,0.041109,0.038062,0.034993,0.040441,0.038348,0.038590,VBENBARO
1,0.878540,0.890066,0.876172,0.886581,0.884776,0.883227,CCMEWNKY
2,0.050810,0.052833,0.053842,0.051593,0.049886,0.051793,VK3KGA9M
3,0.022850,0.022985,0.022714,0.023633,0.021189,0.022674,TT8RPZVC
4,0.022703,0.022362,0.023472,0.024481,0.022216,0.023047,SHQZEYTZ


In [42]:
all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
all_features.reset_index(inplace=True)
important_features = list(all_features[all_features['importance']!=0]['feature'])
all_features

,feature,importance
0,unique_of_Region_Code_Avg_Account_Balance_Bins...,6.633275
1,sum_of_Avg_Account_Balance_per_Credit_Product_...,4.875253
2,std_of_Avg_Account_Balance_per_Credit_Product,3.218597
3,Credit_Product_Channel_Code,3.113593
4,Occupation_Age_Bins_counts,2.927466
...,...,...
147,unique_of_Credit_Product_Occupation_Age_Bins_p...,0.060863
148,unique_of_Region_Code_Gender_per_Credit_Produc...,0.057073
149,unique_of_Credit_Product_Occupation_Avg_Accoun...,0.045416
150,unique_of_Is_Active_Region_Code_per_Occupation...,0.040519


In [41]:
df_submission = final_prediction[['ID', 'Is_Lead']]
df_submission.to_csv("cb_v3_imp_feat.csv", index = False)
df_submission.head()

,ID,Is_Lead
0,VBENBARO,0.038590
1,CCMEWNKY,0.883227
2,VK3KGA9M,0.051793
3,TT8RPZVC,0.022674
4,SHQZEYTZ,0.023047
